In [2]:
# Install necessary libraries
!pip install moviepy openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 65.5 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
from moviepy.editor import VideoFileClip
import re

In [4]:
# Load video
video = VideoFileClip('DCR overview.mp4')

In [5]:
from openai import OpenAI

client = OpenAI(api_key='YOUR OPENAI KEY')

def transcribe_audio(video):
    # Extract audio
    audio_file_path = 'video_audio.wav'
    video.audio.write_audiofile(audio_file_path)

    # Transcribe using OpenAI
    with open(audio_file_path, 'rb') as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )
    return transcription.segments

audio_transcription = transcribe_audio(video)

MoviePy - Writing audio in video_audio.wav


MoviePy - Done.


In [6]:
# Extract text segments from JSON output
text_segments = [{'text': segment['text'], 'start': segment['start'], 'end': segment['end']} for segment in audio_transcription]
print(text_segments)

[{'text': ' The first report is for all devices, including CTV.', 'start': 0.0, 'end': 12.239999771118164}, {'text': ' Looking at the pivot table for April, the total reach was 2.3 billion, so 2,370,069,038,', 'start': 12.239999771118164, 'end': 25.239999771118164}, {'text': ' and this is for the whole audience.', 'start': 25.239999771118164, 'end': 28.399999618530273}, {'text': ' For 18+, it was 2,363,666,772.', 'start': 28.399999618530273, 'end': 36.959999084472656}, {'text': ' For May, 18+, it was 2,421,175,412, and the total reach for May was 2,427,415,712.', 'start': 36.959999084472656, 'end': 56.08000183105469}, {'text': ' If we look at watch time, instead of reach, we see the total watch time for 18+, in April,', 'start': 56.08000183105469, 'end': 68.87999725341797}, {'text': ' was 17,858,883,395, and for May, it was 18,230,148,949.', 'start': 68.87999725341797, 'end': 81.68000030517578}, {'text': ' We can repeat the same exercise for CTV only, so for CTV only, for 18+, the tota

In [7]:
# Combine related segments where one contains conditions and another contains values
def combine_segments(text_segments):
    combined_segments = []
    i = 0
    while i < len(text_segments):
        segment = text_segments[i]
        if i + 1 < len(text_segments) and re.search(r'\d', text_segments[i + 1]['text']):
            # Combine current segment with the next one if it contains a value
            combined_text = segment['text'] + ' ' + text_segments[i + 1]['text']
            combined_segment = {'text': combined_text, 'start': segment['start'], 'end': text_segments[i + 1]['end']}
            combined_segments.append(combined_segment)
            i += 2
        else:
            combined_segments.append(segment)
            i += 1
    return combined_segments

combined_segments = combine_segments(text_segments)
print(combined_segments)


[{'text': ' The first report is for all devices, including CTV.  Looking at the pivot table for April, the total reach was 2.3 billion, so 2,370,069,038,', 'start': 0.0, 'end': 25.239999771118164}, {'text': ' and this is for the whole audience.  For 18+, it was 2,363,666,772.', 'start': 25.239999771118164, 'end': 36.959999084472656}, {'text': ' For May, 18+, it was 2,421,175,412, and the total reach for May was 2,427,415,712.  If we look at watch time, instead of reach, we see the total watch time for 18+, in April,', 'start': 36.959999084472656, 'end': 68.87999725341797}, {'text': ' was 17,858,883,395, and for May, it was 18,230,148,949.  We can repeat the same exercise for CTV only, so for CTV only, for 18+, the total reach', 'start': 68.87999725341797, 'end': 100.87999725341797}, {'text': ' was 2,920,230,254 for April, and for May, it was 2,960,705,540.  Total watch time for CTV in May was 2,55,215,486,163, and for May, it was 2,57,339,257,587.', 'start': 100.87999725341797, 'end': 

In [8]:
# Function to filter out segments that do not contain numbers
def filter_segments_with_numbers(segments):
    filtered_segments = []
    for segment in segments:
        text = segment['text'].strip()
        # Keep segments that contain at least one digit
        if re.search(r'\d', text):
            filtered_segments.append(segment)
    return filtered_segments

filtered_segments = filter_segments_with_numbers(combined_segments)
print(filtered_segments)

[{'text': ' The first report is for all devices, including CTV.  Looking at the pivot table for April, the total reach was 2.3 billion, so 2,370,069,038,', 'start': 0.0, 'end': 25.239999771118164}, {'text': ' and this is for the whole audience.  For 18+, it was 2,363,666,772.', 'start': 25.239999771118164, 'end': 36.959999084472656}, {'text': ' For May, 18+, it was 2,421,175,412, and the total reach for May was 2,427,415,712.  If we look at watch time, instead of reach, we see the total watch time for 18+, in April,', 'start': 36.959999084472656, 'end': 68.87999725341797}, {'text': ' was 17,858,883,395, and for May, it was 18,230,148,949.  We can repeat the same exercise for CTV only, so for CTV only, for 18+, the total reach', 'start': 68.87999725341797, 'end': 100.87999725341797}, {'text': ' was 2,920,230,254 for April, and for May, it was 2,960,705,540.  Total watch time for CTV in May was 2,55,215,486,163, and for May, it was 2,57,339,257,587.', 'start': 100.87999725341797, 'end': 

In [29]:
def extract_watch_time_metrics_and_generate_sql(text_segments):
    sql_queries = []
    example_query = """
    Base SQL Queries:
    SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-04') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';

    SQL Schema:
      start_date          STRING
      timezone            STRING
      country             STRING
      platform            STRING
      min_sec             INT64
      partner             STRING
      watch_time_min      DOUBLE
      views               DOUBLE
    """

    for segment in text_segments:
        text = segment['text']
        start = segment['start']
        end = segment['end']

        user_message = f"""
        Extract the total watch time value in number format without any descriptions and generate an SQL query based on the following text segment from a transcribed video.

        Text Segment:
        {text}

        {example_query}

        Key Points:
          -Use the default query, but if the conditions in the Schema are mentioned in the text segment, update that part. Don't add any extra WHERE conditions.
          -Convert CTV to TV, All to DESKTOP_MOBILE_TABLET_TV.
          -SQL operators should be UPPERCASE.

        Output Format:
        Value: [total watch time value in number format without any descriptions], SQL Query: [Generated SQL query]
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that extracts metrics and generates SQL queries. Ensure the SQL queries are accurate, based on the given schema."},
                    {"role": "user", "content": user_message},
                ]
            )

            result = response.choices[0].message.content
            sql_queries.append({'text': result, 'start': start, 'end': end})
        except Exception as e:
            sql_queries.append({'text': f"Error: {str(e)}", 'start': start, 'end': end})

    return sql_queries


In [30]:
def extract_reach_metrics_and_generate_sql(text_segments):
    sql_queries = []
    example_query = """
     Base SQL Queries:
     select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401
     - start_date (INT64): Date in the format 2024MM01, without quotes
    - country_code (STRING)
    - device (STRING)
    - age_range (STRING)
    - gender (STRING)
    - editorial_reach (INT64)
     """

    for segment in text_segments:
        text = segment['text']
        start = segment['start']
        end = segment['end']

        user_message = f"""
        Extract the total reach value in number format without any descriptions and generate an SQL query based on the following text segment from a transcribed video.
        Text Segment:
        {text}

        {example_query}

        Key Points:
          -use age_range with the format '0-99', min_age = 0, max_age = 99
          -if age_range is 18-99 then would simply put age_range `NOT IN ('0-17')` etc
          -Use the default query, but if the conditions in the Schema are mentioned in the text segment, update that part. Don't add any extra WHERE conditions.
          -SQL operators should be UPPERCASE.


        Output Format:
        Value: [total reach value in number format without any descriptions], SQL Query: [Generated SQL query]
        """

        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that extracts metrics and generates SQL queries. Ensure the SQL queries are accurate, based on the given schema."},
                    {"role": "user", "content": user_message},
                ]
            )

            result = response.choices[0].message.content
            sql_queries.append({'text': result, 'start': start, 'end': end})
        except Exception as e:
            sql_queries.append({'text': f"Error: {str(e)}", 'start': start, 'end': end})

    return sql_queries

In [31]:
# Extract watch time metrics and generate SQL queries
watch_time_sql_queries = extract_watch_time_metrics_and_generate_sql(filtered_segments)
print(watch_time_sql_queries)

[{'text': "Since the text segment doesn't provide a total watch time value and doesn't specify any conditions that were not in the original SQL query, I’ll use the query as it is. Although the segment mentions April, the original query already takes this into account. It also mentions all devices including CTV which is already accounted for in the original query with 'DESKTOP_MOBILE_TABLET_TV'.\n\nSQL Query:\n```sql\nSELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-04') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';\n```\n\nAs the text segment does not provide any specific total watch time value, it's not possible to extract this information. Confirmed outputs would need the actual execution of the SQL query for a specific database.\n\nOutput Format: \nValue: [Not Provided], SQL Query: [Provided Above]", 'start': 0.0, 'end': 2

In [32]:
# Extract reach metrics and generate SQL queries
reach_sql_queries = extract_reach_metrics_and_generate_sql(filtered_segments)
print(reach_sql_queries)


[{'text': "Value: 2370069038, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", 'start': 0.0, 'end': 25.239999771118164}, {'text': "Value: 2363666772, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", 'start': 25.239999771118164, 'end': 36.959999084472656}, {'text': "Value: 2427415712, SQL Query: SELECT SUM(editorial_reach) as editorial_reach FROM ads_people_metrics.yt_reach_artifacts.2024 WHERE country_code = 'US' AND device = 'CTV' AND model LIKE 'quito_%' AND start_date = 20240501 AND age_range NOT IN ('0-17')", 'start': 36.959999084472656, 'end': 68.87999725341797}, {'text': "Value: 17858883395 \nSQL Query: select sum(ed

In [33]:
# Simplified validation function for SQL queries
def validate_sql_query(query):
    # Check if the query contains both "SQL Query:" and starts with "Value:"
    return "SQL Query:" in query and query.strip().startswith("Value:")

In [34]:
# Filter out invalid queries for watch time
valid_watch_time_sql_queries = [query for query in watch_time_sql_queries if validate_sql_query(query['text'])]
print(valid_watch_time_sql_queries)

[{'text': "Value: 2363666772, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-04') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';", 'start': 25.239999771118164, 'end': 36.959999084472656}, {'text': "Value: 2421175412, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-05') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';", 'start': 36.959999084472656, 'end': 68.87999725341797}, {'text': "Value: 18,230,148,949, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-05') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND 

In [35]:
# Filter out invalid queries for reach
valid_reach_sql_queries = [query for query in reach_sql_queries if validate_sql_query(query['text'])]
print(valid_reach_sql_queries)

[{'text': "Value: 2370069038, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", 'start': 0.0, 'end': 25.239999771118164}, {'text': "Value: 2363666772, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", 'start': 25.239999771118164, 'end': 36.959999084472656}, {'text': "Value: 2427415712, SQL Query: SELECT SUM(editorial_reach) as editorial_reach FROM ads_people_metrics.yt_reach_artifacts.2024 WHERE country_code = 'US' AND device = 'CTV' AND model LIKE 'quito_%' AND start_date = 20240501 AND age_range NOT IN ('0-17')", 'start': 36.959999084472656, 'end': 68.87999725341797}, {'text': "Value: 17858883395 \nSQL Query: select sum(ed

In [40]:
def extract_sql_from_text(text):
    match = re.search(r'SQL Query:\s*```sql\n(.*?)\n```', text, re.IGNORECASE | re.DOTALL)
    if not match:
        match = re.search(r'SQL Query:\s*(.*)', text, re.IGNORECASE)
    if match:
        sql_query = match.group(1).strip()
        # Ensure the query is either SUM or MAX type
        if re.match(r'select\s+(sum|max)\(\w+\)', sql_query, re.IGNORECASE):
            return sql_query
    return None
def extract_value_from_text(text):
    match = re.search(r'Value:\s*([\d,]+)', text)
    if match:
        return match.group(1).replace(',', '')
    return None

In [43]:
# Extract distinct SQL queries and their corresponding values
distinct_queries = {}
for query in valid_watch_time_sql_queries:
    original_sql = extract_sql_from_text(query['text'])
    value = extract_value_from_text(query['text'])
    if original_sql and value:
        normalized_sql = original_sql.lower()
        if normalized_sql not in distinct_queries:
            distinct_queries[normalized_sql] = {'original_sql': original_sql, 'value': value}

# Export distinct SQL queries and their values to a text file
with open('watch_time_sql_queries.txt', 'w') as file:
    for query in distinct_queries.values():
        file.write(f"Value: {query['value']}, SQL Query: {query['original_sql']}\n")

# Print the unique queries and values
unique_watch_queries = [f"Value: {query['value']}, SQL Query: {query['original_sql']}" for query in distinct_queries.values()]
print(unique_watch_queries)

["Value: 2363666772, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-04') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';", "Value: 2421175412, SQL Query: SELECT MAX(watch_time_min) AS watch_time_min FROM yt_mp_relaylog_data.FirstPartyWatchViews.all WHERE start_date IN ('2024-05') AND platform = 'DESKTOP_MOBILE_TABLET_TV' AND country = 'US' AND partner = 'NIELSEN' AND min_sec = 0 AND timezone = 'America/New_York';"]


In [44]:
# Extract distinct SQL queries and their corresponding values
distinct_queries = {}
for query in valid_reach_sql_queries:
    original_sql = extract_sql_from_text(query['text'])
    value = extract_value_from_text(query['text'])
    if original_sql and value:
        normalized_sql = original_sql.lower()
        if normalized_sql not in distinct_queries:
            distinct_queries[normalized_sql] = {'original_sql': original_sql, 'value': value}

# Export distinct SQL queries and their values to a text file
with open('reach_sql_queries.txt', 'w') as file:
    for query in distinct_queries.values():
        file.write(f"Value: {query['value']}, SQL Query: {query['original_sql']}\n")

# Print the unique queries and values
unique_reach_queries = [f"Value: {query['value']}, SQL Query: {query['original_sql']}" for query in distinct_queries.values()]
print(unique_reach_queries)

["Value: 2370069038, SQL Query: select sum(editorial_reach) as editorial_reach from ads_people_metrics.yt_reach_artifacts.2024 where country_code = 'US' and device = 'CTV' and model like 'quito_%' AND start_date = 20240401 and age_range NOT IN ('0-17')", "Value: 2427415712, SQL Query: SELECT SUM(editorial_reach) as editorial_reach FROM ads_people_metrics.yt_reach_artifacts.2024 WHERE country_code = 'US' AND device = 'CTV' AND model LIKE 'quito_%' AND start_date = 20240501 AND age_range NOT IN ('0-17')"]
